#### Introduction et objectifs du projet

##### <u>Contexte :</u>

L'analyse de sentiments est un domaine essentiel en science des données, en particulier en NLP. Elle utilise le machine learning et la linguistique pour détecter, extraire et étudier les émotions et les informations subjectives dans les textes sources.

Dans le contexte des avis de films, l'analyse de sentiments permet de comprendre les perceptions et les émotions des spectateurs à partir de leurs commentaires, ce qui est précieux pour les critiques de films, les studios de cinéma et les plateformes de streaming.

##### <u>Objectifs :</u>

Ce projet vise à développer une application interactive en Python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques. Les objectifs de l'application sont les suivants :

1. Permettre aux utilisateurs de saisir un titre de film ou son ID pour récupérer des avis pertinents.

2. Analyser les sentiments exprimés dans ces avis, en calculant la polarité (positivité ou négativité des sentiments) et la subjectivité (mesure de l'opinion personnelle par rapport à un fait objectif) des commentaires.

3. Comparer les "ratings" (notes) des films avec la polarité des avis pour voir si les évaluations numériques correspondent aux sentiments exprimés dans les commentaires.

4. Examiner les différences, le cas échéant, entre les sentiments exprimés avant et après la sortie du film, fournissant ainsi une perspective unique sur l'évolution des perceptions des spectateurs.

##### <u>Méthodes de récupération des données :</u>

Dans un premier temps, pour éviter de surcharger l'API de TMDB et de devoir gérer des bases de données trop volumineuses, nous avons décidé de limiter notre étude aux films sortis entre x et y. Une fois cette sélection effectuée, nous avons procédé à la récupération des données via 3 méthodes différentes :

- L'API de TMDB : "https://developers.themoviedb.org/3/"
- Scraping du site : imdb.com (via la librairie python : beautifulsoup4)
- Utilisation de la librairie python : imdbpy

L'objectif de cette section est de comparer les différentes méthodes de récupération des données afin de déterminer laquelle est la plus efficace et la plus pertinente pour notre projet.
Dans une première approche il était question de tester les 3 méthodes, voir les avantages et inconvénients de chacune d'entre elles, de savoir si elles étaient complémentaires ou non, et de déterminer laquelle était la plus efficace et la plus pertinente pour notre projet.

Dans un second temps l'objectif est de structurer les données récupérées afin de pouvoir les exploiter dans la suite du projet. Si possible par une seule des méthodes de récupération des données mentionnées ci-dessus.

Ensuite, il s'agira de mettre en place une méthode d'analyse de sentiments sur les commentaires récupérés, il existe des librairies python qui permettent de faire cela, mais il est également possible de créer notre propre méthode d'analyse de sentiments. 

Choix numéro 1 : Réaliser nos propres méthodes d'analyse de sentiments.

Choix numéro 2 : Utiliser des librairies python existantes tel que : TextBlob, NLTK.

Enfin il s'agira de mettre en place une application interactive en python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques.


#### <u>Import des librairies</u>

- package imdb à importer via le code ci-dessous : (important qu'il soit installé sur la machine et qu'il soit à jour)

In [1]:
#pip install git+https://github.com/santhoshse7en/imdb

In [2]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import requests
from datetime import datetime
import json
import re
import pandas as pd

- Appel de la clé API de TMDB 

In [3]:
with open('clefAPI.json', 'r') as config_file:
    config = json.load(config_file)

api_key = config['tmdb']['api_key']

#### <u>Méthode 1 : scraping du site imdb.com </u>

Première étape : on va tester notre procédé sur un seul film, pour voir si ça fonctionne, et si oui, on pourra ensuite l'appliquer à tous les films de notre base de données.

Étape crucial : on doit récupérer la date de sortie du film, pour pouvoir ensuite récupérer les commentaires avant et après la sortie du film. 

The Shawshank Redemption : ID sur le site de TMDB : "tt0111161"

In [4]:
film_id = 'tt0111161'

url_date_release = f"https://api.themoviedb.org/3/movie/{film_id}?api_key={api_key}&language=en-US"

response = requests.get(url_date_release)

if response.status_code == 200:
    film_details = response.json()
    date_sortie_str = film_details.get("release_date", "")
    if date_sortie_str:
        date_sortie = datetime.strptime(date_sortie_str, "%Y-%m-%d")
        print(f"Date de sortie du film : {date_sortie}")
    else:
        print("Date de sortie non trouvée")
else:
    print("Erreur lors de la requête :", response.status_code)

url_film = f"https://www.imdb.com/title/{film_id}/reviews" # Récupérer les avis sur IMDb

response = requests.get(url_film)

reviews_dict = {}

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    review_elements = soup.find_all('div', class_='review-container')
    for element in review_elements:
        review_text = element.find('div', class_='text show-more__control').get_text().strip()
        review_date_str = element.find('span', class_='review-date').get_text().strip()
        review_date = datetime.strptime(review_date_str, '%d %B %Y').date()
        reviews_dict[review_date] = review_text
else:
    print("Erreur lors de la requête :", response.status_code)

print("Nombre d'avis pour ce film :",len(reviews_dict)) 

Date de sortie du film : 1994-09-23 00:00:00
Nombre d'avis pour ce film : 25


- Objectif : voir si on a assez de commentaires avant et après la sortie du film pour pouvoir faire une analyse de sentiments.

In [5]:
avis_avant = {}
avis_apres = {}

for review_date, review_text in reviews_dict.items():
    if review_date < date_sortie.date():
        avis_avant[review_date] = review_text
    else:
        avis_apres[review_date] = review_text

print("Nombre d'avis avant la sortie du film :", len(avis_avant))
print("Nombre d'avis après la sortie du film :", len(avis_apres))

Nombre d'avis avant la sortie du film : 0
Nombre d'avis après la sortie du film : 25


On constate alors qu'avant la sortie du film, on a 0 commentaires, et après la sortie du film, on a 25 commentaire. On ne peut donc pas faire d'analyse de sentiments sur ce film. Et en testant sur d'autres films, on se rend compte que c'est le cas pour tous des films de notre base de données. Afin de réaliser le projet dans son intégralité, il faudrait donc récupérer les commentaires sur une autre plateforme que TMDB. Toutefois nous manquons de temps et d'expérience pour réaliser cela. Nous allons donc nous contenter de réaliser une analyse de sentiments sur les commentaires récupérés sur TMDB.

- Affichage des 5 premiers commentaires récupérés sur TMDB pour ce premier film :

In [6]:
print("Avis après la sortie:")
for date, review in list(avis_apres.items())[:5]:
    print(f"{date}: {review}\n")

Avis après la sortie:
2010-07-24: The Shawshank Redemption is written and directed by Frank Darabont. It is an adaptation of the Stephen King novella Rita Hayworth and Shawshank Redemption. Starring Tim Robbins and Morgan Freeman, the film portrays the story of Andy Dufresne (Robbins), a banker who is sentenced to two life sentences at Shawshank State Prison for apparently murdering his wife and her lover. Andy finds it tough going but finds solace in the friendship he forms with fellow inmate Ellis "Red" Redding (Freeman). While things start to pick up when the warden finds Andy a prison job more befitting his talents as a banker. However, the arrival of another inmate is going to vastly change things for all of them.There was no fanfare or bunting put out for the release of the film back in 94, with a title that didn't give much inkling to anyone about what it was about, and with Columbia Pictures unsure how to market it, Shawshank Redemption barely registered at the box office. Howe

#### <u>Méthode 2 : utilisation de l'API </u>

In [7]:
def get_movies_between_dates(start_date, end_date, api_key):
    base_url = "https://api.themoviedb.org/3/discover/movie"
    url = f"{base_url}?api_key={api_key}&primary_release_date.gte={start_date}&primary_release_date.lte={end_date}&region=FR"
    response = requests.get(url)
    if response.status_code == 200:
        movies_data = response.json()['results']
        return [(movie['id'], movie['title']) for movie in movies_data]
    else:
        print("Erreur lors de la requête :", response.status_code)
        return []
    
start_date = "2022-01-01" # FONCTIONNE ! On peut donc définir la période de recherche que l'on souhaite
end_date = "2022-01-31" 

print(f"Recherche des films sortis entre le {start_date} et le {end_date}", end=" ")
print(get_movies_between_dates(start_date, end_date, api_key))

Recherche des films sortis entre le 2022-01-01 et le 2022-01-31 [(899082, 'Harry Potter 20th Anniversary: Return to Hogwarts'), (646385, 'Scream'), (778810, 'Fireheart'), (774825, 'The Ice Age Adventures of Buck Wild'), (557946, 'Last Looks'), (575322, 'Project Gemini'), (926899, 'The House'), (854867, 'Werewolf Castle'), (522016, 'The 355'), (698508, 'Redeeming Love'), (803114, 'The Requin'), (760926, 'Gold'), (760336, 'Munich: The Edge of War'), (916740, 'Brazen'), (790142, 'The Royal Treatment'), (689700, 'Clean'), (591120, 'The Pirates: The Last Royal Treasure'), (844398, 'Shattered'), (661791, 'La Abuela (The Grandmother)'), (803104, 'Sex Appeal')]


#### <u>Méthode 3 : utilisation de la librairie python : imdbpy</u>

In [8]:
# from cinemagoer import IMDb
# ia = IMDb()
# movie = ia.get_movie('0111161')  # Pour 'The Shawshank Redemption'
# print(movie['title'])  # Afficher le titre du film
!pip show IMDbPY

Name: IMDbPY
Version: 2022.7.9
Summary: IMDbPY is now cinemagoer
Home-page: https://cinemagoer.github.io/
Author: 
Author-email: 
License: 
Location: C:\Users\pc\anaconda3\Lib\site-packages
Requires: cinemagoer
Required-by: 


#### <u>Récupération des données via la méthode de scraping du site imdb.com</u>

In [9]:
def get_imdb_reviews(movie_id): # FONCTIONNE
    url = f"https://www.imdb.com/title/{movie_id}/reviews"
    response = requests.get(url)
    reviews = {}
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        review_elements = soup.find_all('div', class_='review-container')
        if not review_elements:
            print(f"Aucune critique trouvée pour {movie_id}")
            return reviews
        for element in review_elements:
            review_text = element.find('div', class_='text show-more__control').get_text().strip()
            review_date = datetime.strptime(element.find('span', class_='review-date').get_text(), '%d %B %Y')
            reviews[review_date] = review_text
    else:
        print(f"Erreur lors de la récupération des avis pour {movie_id}: {response.status_code}")
    return reviews

print(get_imdb_reviews('tt0111161'))

{datetime.datetime(2010, 7, 24, 0, 0): 'The Shawshank Redemption is written and directed by Frank Darabont. It is an adaptation of the Stephen King novella Rita Hayworth and Shawshank Redemption. Starring Tim Robbins and Morgan Freeman, the film portrays the story of Andy Dufresne (Robbins), a banker who is sentenced to two life sentences at Shawshank State Prison for apparently murdering his wife and her lover. Andy finds it tough going but finds solace in the friendship he forms with fellow inmate Ellis "Red" Redding (Freeman). While things start to pick up when the warden finds Andy a prison job more befitting his talents as a banker. However, the arrival of another inmate is going to vastly change things for all of them.There was no fanfare or bunting put out for the release of the film back in 94, with a title that didn\'t give much inkling to anyone about what it was about, and with Columbia Pictures unsure how to market it, Shawshank Redemption barely registered at the box offic

#### <u>Récupération des données via l'API de TMDB</u>

In [11]:
def get_movie_details(movie_id, api_key):
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Erreur lors de la requête :", response.status_code)
        return None

print(get_movie_details('tt0111161', api_key))

{'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'belongs_to_collection': None, 'budget': 25000000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}], 'homepage': '', 'id': 278, 'imdb_id': 'tt0111161', 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 110.352, 'poster_path': '/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg', 'production_companies': [{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxYqAlPPykp.png', 'name': 'Castle Rock Entertainment', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'Unit

In [12]:
def get_movie_details_and_reviews(movie_id, api_key):
    base_url = "https://api.themoviedb.org/3/movie/"
    url = f"{base_url}{movie_id}?api_key={api_key}"
    response = requests.get(url)
    movie_details = response.json() if response.status_code == 200 else {}
    reviews_url = f"{base_url}{movie_id}/reviews?api_key={api_key}"
    reviews_response = requests.get(reviews_url)
    reviews_data = reviews_response.json() if reviews_response.status_code == 200 else {}

    return movie_details, reviews_data


print(get_movie_details_and_reviews('tt0111161', api_key))

({'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'belongs_to_collection': None, 'budget': 25000000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 80, 'name': 'Crime'}], 'homepage': '', 'id': 278, 'imdb_id': 'tt0111161', 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 110.352, 'poster_path': '/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg', 'production_companies': [{'id': 97, 'logo_path': '/7znWcbDd4PcJzJUlJxYqAlPPykp.png', 'name': 'Castle Rock Entertainment', 'origin_country': 'US'}], 'production_countries': [{'iso_3166_1': 'US', 'name': 'Uni

#### <u>On transforme nos données en json</u>

In [13]:
movies_data = get_movies_between_dates(start_date, end_date, api_key)
movies_details = {}

for movie_id, movie_title in movies_data:
    movie_details, reviews_data = get_movie_details_and_reviews(movie_id, api_key)
    movies_details[movie_id] = {
        'title': movie_title,
        'details': movie_details,
        'reviews': reviews_data
    }

# Ne lancer qu'une seule fois pour éviter d'écraser le fichier JSON

#with open('movies_details.json', 'w') as json_file: 
#    json.dump(movies_details, json_file, indent=4)

Maintenant que nous avons récupérées toutes les données utiles à notre projet dans un fichier "movies_details.json", nous allons pouvoir les exploiter.

In [22]:
movies_details

{'646385': {'title': 'Scream',
  'details': {'adult': False,
   'backdrop_path': '/ifUfE79O1raUwbaQRIB7XnFz5ZC.jpg',
   'belongs_to_collection': {'id': 2602,
    'name': 'Scream Collection',
    'poster_path': '/p3EjClFy20jjT0u06dzBs4lvvhi.jpg',
    'backdrop_path': '/oUcscMECv8DOBsAPCh3KnDZqAC4.jpg'},
   'budget': 24000000,
   'genres': [{'id': 27, 'name': 'Horror'},
    {'id': 9648, 'name': 'Mystery'},
    {'id': 53, 'name': 'Thriller'}],
   'homepage': 'https://www.paramountmovies.com/movies/scream',
   'id': 646385,
   'imdb_id': 'tt11245972',
   'original_language': 'en',
   'original_title': 'Scream',
   'overview': 'Twenty-five years after a streak of brutal murders shocked the quiet town of Woodsboro, a new killer has donned the Ghostface mask and begins targeting a group of teenagers to resurrect secrets from the town’s deadly past.',
   'popularity': 81.454,
   'poster_path': '/971Kqs1q4nuSc9arn1QAuKYbfxy.jpg',
   'production_companies': [{'id': 4,
     'logo_path': '/gz66EfN

Avant de se lancer dans l'analyse de sentiments, nous allons d'abord faire des vérifications sur les données inscritent dans notre fichier json. Nous allons requêter ces données pour voir si elles sont bien présentes et si elles sont bien au format attendu.

On peut par exemple demander le nombre de films dans notre json :

In [14]:
with open('movies_details.json', 'r') as json_file:
    movies_details = json.load(json_file)

num_films = len(movies_details)

print("Nombres de  films:", num_films)

Nombres de  films: 20


On peut également demander le nombre d'avis pour chaque film :

In [18]:
import json

try:
    with open('movies_details.json', 'r') as file:
        movies_details = json.load(file)
    reviews_count = {}
    for movie in movies_details.values():
        num_reviews = len(movie['reviews']['results'])
        movie_id = movie['reviews']['id']
        reviews_count[movie_id] = num_reviews
    for movie_id, count in reviews_count.items():
        print(f"Film ID {movie_id}: Nombre d'avis = {count}")
except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")


Film ID 646385: Nombre d'avis = 9
Film ID 774825: Nombre d'avis = 1
Film ID 557946: Nombre d'avis = 2
Film ID 899082: Nombre d'avis = 0
Film ID 522016: Nombre d'avis = 6
Film ID 575322: Nombre d'avis = 0
Film ID 245842: Nombre d'avis = 3
Film ID 803114: Nombre d'avis = 5
Film ID 926899: Nombre d'avis = 0
Film ID 916740: Nombre d'avis = 1
Film ID 854867: Nombre d'avis = 1
Film ID 825808: Nombre d'avis = 0
Film ID 591120: Nombre d'avis = 1
Film ID 844398: Nombre d'avis = 1
Film ID 760336: Nombre d'avis = 2
Film ID 698508: Nombre d'avis = 1
Film ID 760926: Nombre d'avis = 1
Film ID 778810: Nombre d'avis = 0
Film ID 661791: Nombre d'avis = 0
Film ID 800425: Nombre d'avis = 0


Maintenant on test d'afficher les avis sur un film en particulier : je prends par exemple l'ID du film x : "646385" et je récupère les avis sur ce film :

In [23]:
fichier_json = 'movies_details.json'
movie_id_to_check = 646385

try:
    with open(fichier_json, 'r') as file:
        movies_details = json.load(file)

    selected_movie = movies_details[str(movie_id_to_check)]
    reviews = selected_movie['reviews']['results']
    
    for review in reviews:
        print(f"Auteur : {review['author']}")
        print(f"Rating : {review['author_details']['rating']}")
        print(f"Contenu : {review['content']}\n")

except Exception as e:
    print(f"Erreur lors de la lecture du fichier JSON: {e}")

Auteur : garethmb
Rating : None
Contenu : It is hard to believe it has been 25 years since “Scream” premiered and in doing so; brought new life into the slasher genre which had become stale due to an abundance of direct to DVD and cable releases combined with many uninspired and stale theatrical releases.

The movie was a massive success and launched a very successful trilogy and made the Ghostface killer and many moments from the series Pop Culture mainstays.

After a hiatus; “Scream 4” arrived and continued the series and in 2022; “Scream” has arrived which continues the story to the eager anticipation of fans.

When a young girl named Tara (Jenna Ortega); is brutally attacked in a fashion very similar to the Ghostface Killer; her sister Sam (Melissa Barrera); returns to Woodsboro to be by her side while Tara recuperates.

Sam is joined by her boyfriend Richie (Jack Quaid), who has no idea about the history of Woodsboro nor the popular “Stab” film series that is based on the events o

## Idées pour la suite du projet :

- Créer l'application interactive en python permettant aux utilisateurs de demander des analyses de sentiments sur des films spécifiques.
- Analyser les mots clés les plus utilisés dans les commentaires récupérés sur TMDB.
- Faire une analyse de sentiments sur les commentaires récupérés sur TMDB.
- Faire une moyenne de notes de sentiments pour chaque film.
- Comparer les notes de sentiments des avis avec les notes de films attribuées par les utilisateurs.


#### <u> Création de l'application interactive</u>

In [33]:
from flask import Flask, render_template, request
import json

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    with open('movies_details.json', 'r') as file:
        movies_details = json.load(file)
    
    movies_list = [(movie['reviews']['id'], movie['title']) 
                for movie in movies_details.values() 
                if 'reviews' in movie and 'id' in movie['reviews'] and 'title' in movie]

    return render_template('index.html', movies_list=movies_list)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-12-16 17:05:21,767] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "c:\Users\pc\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
  File "c:\Users\pc\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
  File "c:\Users\pc\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
  File "c:\Users\pc\anaconda3\Lib\site-packages\flask\app.py", line 1796, in dispatch_request
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_4804\4245691928.py", line 16, in index
    return render_template('index.html', movies_list=movies_list)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pc\anaconda3\Lib\site-packages\flask\templating.py", line 146, in render_template
    :param template_name_or_list: The name of the template to render. If
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F

#### <u> Analyse des mots clés les plus utilisés dans les commentaires récupérés sur TMDB</u>

#### <u> Analyse de sentiments sur les commentaires récupérés sur TMDB</u>